In [ ]:
# https://lovit.github.io/nlp/2019/02/16/logistic_w2v_ner/

In [1]:
import re
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
from konlpy.tag import Okt

In [3]:
okt = Okt()
print(okt.morphs("아이유랑 비슷한 노래 추천해줘"))

['아이유', '랑', '비슷한', '노래', '추천', '해줘']


In [4]:
f =  open("text_data_ko.txt", 'r')

data = f.read()

f.close()

In [ ]:
# data = data[:10000]

In [5]:
data = data.split('\n')

In [ ]:
len(data)

In [ ]:
# import pickle
# with open('words.bin', 'rb') as f:
#     words = pickle.load(f)

In [6]:
from tqdm.notebook import tqdm

In [ ]:
# content = []
# for d in tqdm(data):
#     content.append(okt.morphs(d))

In [7]:
import pickle

In [ ]:
#저장
# with open('tokenizedData.bin', 'wb') as f:
#     pickle.dump(content, f)

In [8]:
# with open('tokenizedData.bin', 'rb') as f:
#     content = pickle.load(f)

In [10]:
for i in tqdm(range(3990459,len(data))):
    content.append(okt.morphs(data[i]))

In [ ]:
len(data)

In [11]:
len(content)
# content[1]

5948717

In [ ]:
# content = words
# len(content)

In [12]:
from collections import Counter
from collections import defaultdict

In [13]:
countDict = defaultdict(int)
for sentence in content:
    for word in sentence:
        countDict[word]+=1
countDict

defaultdict(int,
            {'락': 30341,
             '노래': 3289714,
             '추천': 2002295,
             '해줘': 1984085,
             '음악': 1400710,
             '곡': 1227187,
             '알려줘': 1982731,
             '락곡': 6183,
             '락노': 3173,
             '래': 29530,
             '들려줘': 1982773,
             '추억': 70623,
             '회상': 44685,
             '까페': 26358,
             '잔잔한': 95633,
             '연말': 5299,
             '눈': 15233,
             '오는': 32877,
             '날': 54340,
             '날곡': 5655,
             '캐럴': 1017,
             '분위기': 26204,
             '따듯': 1800,
             '한': 56222,
             '크리스마스캐럴': 63,
             '겨울': 56591,
             '크리스마스': 14334,
             '겨울왕국': 477,
             '송': 7078,
             '송곡': 815,
             '댄스': 34116,
             '운동': 26383,
             '드라이브': 93093,
             '트로피': 2436,
             '컬': 2014,
             '하우스': 5242,
             '하우스음악': 1052,
            

In [14]:
import operator
import pandas as pd

In [15]:
countDict = dict(sorted(countDict.items(), key=operator.itemgetter(1), reverse=True))

In [16]:
k = list(countDict.keys())
v = list(countDict.values())
df = pd.DataFrame(v,k).reset_index().reset_index().rename(columns={"level_0":"idx", "index":"vocab", 0:"count"})
# df

In [17]:
idxToVocab = df[['vocab','idx']].to_dict()['vocab']
idxToCount = df[['count','idx']].to_dict()['count']
vocabToIdx = df[['idx','vocab']].set_index('vocab').to_dict()['idx']

In [ ]:
# Feature

In [18]:
def feature_to_idx(i, j, vocab_idx, window, n_terms):
    if j < i:
        return n_terms * (j - i + window) + vocab_idx
    else:
        return n_terms * (j - i + window - 1) + vocab_idx

In [19]:
def idx_to_feature(feature_idx, idx_to_vocab, window):
    # 몫
    position = feature_idx // len(idx_to_vocab)
    if position < window:
        feature = 'X[-%d] = ' % (window - position)
    else:
        feature = 'X[%d] = ' % (position - window + 1)
    # 나머지
    vocab_idx = feature_idx % len(idx_to_vocab)
    feature += idx_to_vocab[vocab_idx]
    return feature

In [20]:
from scipy.sparse import csr_matrix

In [21]:
def create_window_cooccurrence_matrix(vocab_to_idx, sentences, window=2, verbose=True):

    n_terms = len(vocab_to_idx)

    rows = []
    cols = []
    words = []

    row_idx = 0
    col_idx = window * 2 * n_terms

    for i_sent, sent in enumerate(sentences):

        if verbose and i_sent % 10000 == 0:
            print('\rcreating train dataset {} rows from {} sents'.format(row_idx, i_sent), end='')

        n_words = len(sent)

        for i, word in enumerate(sent):
            if not (word in vocab_to_idx):
                continue

            b = max(0, i - window)
            e = min(i + window + 1, n_words)

            features = []
            for j in range(b, e):
                if i == j:
                    continue
                j_idx = vocab_to_idx.get(sent[j], -1)
                if j_idx == -1:
                    continue
                features.append(feature_to_idx(i, j, j_idx, window, n_terms))

            if not features:
                continue

            # sparse matrix element
            for col in features:
                rows.append(row_idx)
                cols.append(col)

            # words element
            words.append(word)

            row_idx += 1

    if verbose:
        print('\rtrain dataset {} rows from {} sents was created    '.format(row_idx, i_sent))

    # to csr matrix
    rows = np.asarray(rows, dtype=np.int)
    cols = np.asarray(cols, dtype=np.int)
    data = np.ones(rows.shape[0], dtype=np.int)
    X = csr_matrix((data, (rows, cols)), shape=(row_idx, col_idx))

    return X, words

In [22]:
window = 2

X, words = create_window_cooccurrence_matrix(
    vocabToIdx, content, window)

train dataset 25650085 rows from 5948716 sents was created    


In [23]:
print(X.shape)

(25650085, 208556)


In [24]:
from gensim.models import word2vec

In [25]:
word2vec = word2vec.Word2Vec.load("word2vec_ko.model")

In [48]:
print(word2vec.most_similar("이효리"))

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('홍진영', 0.9589906930923462), ('잔나비', 0.9502841830253601), ('조성진', 0.9406865835189819), ('검정치마', 0.9317808747291565), ('정준일', 0.9244722127914429), ('젝스키스', 0.9239343404769897), ('빅밴드', 0.9212276935577393), ('인피니트', 0.9197981357574463), ('이유진', 0.9175125360488892), ('정승환', 0.9157751202583313)]


In [49]:
seed_words = {word for word, _ in word2vec.most_similar('아이유', topn=100)}
seed_words.update({word for word, _ in word2vec.most_similar('이효리', topn=100)})

print(len(seed_words)) # 174

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


161


In [50]:
y = np.zeros(X.shape[0], dtype=np.int)
for i, word in enumerate(words):
    if word in seed_words:
        y[i] = 1

y.sum() # 127998

127270

In [29]:
from sklearn.linear_model import LogisticRegression

In [51]:
logistic = LogisticRegression()
logistic.fit(X, y)
y_pred = logistic.predict(X)
y_prob = logistic.predict_proba(X)[:,1]

In [52]:
# word count
word_counter = Counter(words)

# prediction count
pred_pos = defaultdict(int)
for row in np.where(y_prob >= 0.05)[0]:
    pred_pos[words[row]] += 1
pred_pos = {word:pos/word_counter[word] for word, pos in pred_pos.items()}

In [53]:
for word, prob in sorted(pred_pos.items(), key=lambda x:-x[1])[:1000]:
    if word in seed_words:
        continue
    idx = vocabToIdx[word]
    count = idxToCount[idx]

In [54]:
top_instances = np.where(0.7 <= y_prob)[0]
top_probs = y_prob[top_instances]

print(top_instances.shape) # (730,)
print(top_probs.shape) # (730,)

(925,)
(925,)


In [55]:
def get_unique_top_instances(sample_idxs, probs):

    # slice samples
    X_samples = X[sample_idxs]
    rows, cols = X_samples.nonzero()

    # find unique instance
    instance_prob = {}
    instance_count = defaultdict(int)
    before_row = None

    def update_dict(features, prob):
        features = sorted(features, key=lambda x:x[0])
        instance = ', '.join(f[1] for f in features)
        instance_prob[instance] = prob
        instance_count[instance] += 1
        return []

    features = [] # temporal variable
    for row, feature_idx in zip(rows, cols):
        # update unique dictionary
        if row != before_row and features:
            features = update_dict(features, probs[row])
        # update temporal variable
        before_row = row
        feature = idx_to_feature(feature_idx, idxToVocab, window)
        features.append((feature_idx, feature))

    # last elements
    if features:
        update_dict(features, probs[row])

    return instance_prob, instance_count

instance_prob, instance_count = get_unique_top_instances(top_instances, top_probs)    

In [56]:
print(instance_prob)

{'X[1] = 퇴근길, X[2] = 노래': 0.7508870357276866, 'X[1] = 퇴근길, X[2] = 음악': 0.7661901531299259, 'X[1] = 퇴근길, X[2] = 곡': 0.9057066076273053, 'X[-2] = 캘빈, X[-1] = 클라인, X[1] = 노래, X[2] = 추천': 0.7479857635586885, 'X[-2] = 캘빈, X[-1] = 클라인, X[1] = 노래, X[2] = 알려줘': 0.747755615918449, 'X[-2] = 캘빈, X[-1] = 클라인, X[1] = 노래, X[2] = 들려줘': 0.707948919841811, 'X[1] = 입문, X[2] = 곡곡': 0.707948919841811, 'X[-2] = 육성재, X[-1] = (, X[1] = ), X[2] = 노래': 0.7326949810283916, 'X[-2] = 박승화, X[-1] = (, X[1] = ), X[2] = 노래': 0.8952219020883948, 'X[-2] = 건지, X[-1] = (, X[1] = ),, X[2] = 미스티': 0.9452882902166443, 'X[1] = ,, X[2] = 김준규': 0.7803658451151471, 'X[-2] = 박지민, X[-1] = ,, X[1] = ), X[2] = 노래': 0.9289537886161288, 'X[-2] = 윤덕원, X[-1] = (, X[1] = ), X[2] = 노래': 0.8952219020883948, 'X[-2] = 건지, X[-1] = (, X[1] = ), X[2] = 노래': 0.9057066076273053, 'X[1] = ,, X[2] = 유주': 0.8416300236770594, 'X[-2] = 이세준, X[-1] = (, X[1] = ), X[2] = 노래': 0.9026053611607493, 'X[-2] = 임현식, X[-1] = (, X[1] = ), X[2] = 노래': 0.9452882902